读取文件

In [1]:
from transformers import pipeline
summarizer = pipeline("summarization")

In [1]:
from bs4 import BeautifulSoup

In [2]:
blacklist = [   '[document]',   'noscript', 'header',   'html', 'meta', 'head','input', 'script',   ]

In [3]:
import ebooklib
from ebooklib import epub
def epub2thtml(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters.append(item.get_content())
    return chapters

In [4]:
def chap2text(chap):
    output = ''
    soup = BeautifulSoup(chap, 'html.parser')
    text = soup.find_all(text=True)
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    return output

In [5]:
def thtml2ttext(thtml):
    Output = []
    for html in thtml:
        text =  chap2text(html)
        Output.append(text)
    return Output

In [6]:
def epub2text(epub_path):
    chapters = epub2thtml(epub_path)
    ttext = thtml2ttext(chapters)
    return ttext

In [7]:
out=epub2text('./Murder_on_the_Orient_Express.epub')

In [8]:
len(out)

47

In [9]:
part_one = out[5:13]
part_two = out[14:29] 
part_three = out[30:39]

In [10]:
fold_path = "./raw_text/"
def save_chapter_by_part(part, chapters):
    for i in range(len(chapters)):
        file_name = "Part{0}_Chapter{1}".format(part, i+1)
        with open(fold_path+file_name, "w") as f:
            f.write(chapters[i])

In [11]:
save_chapter_by_part(1,part_one)
save_chapter_by_part(2,part_two)
save_chapter_by_part(3,part_three)

### 预处理

#### (1) 大写转小写

In [66]:
example_text = part_one[0]

In [67]:
example_text = example_text.lower()

#### (2) 处理缩写单词，去除标点符号

In [68]:
import nltk
import re
from nltk.tokenize import ToktokTokenizer
def clean_text(text):
#     text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text


In [69]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize

In [70]:
punct = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-—“”’'

In [71]:
def strip_list_noempty(mylist):
    newlist = [item.strip() if hasattr(item, 'strip') else item for item in mylist]
    return [item for item in newlist if item != '']

def clean_punct(text, type = "str"): 
#     words = token.tokenize(text)
    words = word_tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    for w in words:
        punctuation_filtered.append(regex.sub('', w))
    filtered_list = strip_list_noempty(punctuation_filtered)
    if type == "str":
        return ' '.join(map(str, filtered_list))
    else:
        return filtered_list

In [72]:
example_text = clean_text(example_text)

In [73]:
example_words_list = clean_punct(example_text, 'list')

In [74]:
example_text = clean_punct(example_text)

In [99]:
# 在这一步进行NER，帮助筛选数据
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
doc = nlp(example_text)
NE = set(word for X in doc.ents for word in X.text.split())

In [76]:
len(example_words_list)

2520

In [77]:
from collections import Counter
counter_before = Counter(example_words_list)

In [78]:
len(counter_before)

858

### (3) 词型还原

In [79]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [80]:
lemma=WordNetLemmatizer()
def lemitizeWords(text, type="str"):
    words=word_tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    if type == "str":
        return ' '.join(map(str, listLemma))
    else:
        return listLemma

In [81]:
lemma_result_list = lemitizeWords(example_text, type = "list")

In [82]:
example_text = lemitizeWords(example_text)

In [83]:
counter_lemma = Counter(lemma_result_list)

In [84]:
len(counter_lemma)

805

### (4) 去除停顿词

In [85]:
stop_words = set(stopwords.words("english"))
def stopWordsRemove(text, type = "str"):
    stop_words = set(stopwords.words("english"))
    words=word_tokenize(text)
    filtered = [w for w in words if not w in stop_words]
    if type == "str":
        return ' '.join(map(str, filtered))
    else:
        return filtered

In [86]:
remove_stop_result_list = stopWordsRemove(example_text, list)

In [87]:
example_text = stopWordsRemove(example_text)

In [88]:
counter_stop_list = Counter(remove_stop_result_list)

In [89]:
len(counter_stop_list)

706

In [90]:
len(example_text)

8663

In [100]:
word_dic = dict(counter_stop_list)

In [111]:
valuable_word = []
for w, c in word_dic.items():
    if len(w)>5 and c > 2 and w not in NE:
        valuable_word.append(w)

In [112]:
len(valuable_word)

31

In [113]:
valuable_word

['platform', 'lieutenant', 'nothing', 'stranger', 'dubosc', 'course', 'general', 'officer', 'another', 'suddenly', 'conversation', 'glance', 'speech', 'people', 'travel', 'little', 'compartment', 'window', 'conductor', 'monsieur', 'colonel', 'halfpast', 'coffee', 'breakfast', 'rather', 'slightly', 'arbuthnot', 'whether', 'straight', 'perhaps', 'something']